In [1]:
%pip install pandas
%pip install scikit-learn
%pip install imblearn
%pip install numpy
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import csv

# 1. Đọc dữ liệu từ file CSV
data = pd.read_csv('./dataset_en/Amazon_Product_Review_full_en_cleaned.csv')

# Hiển thị các cột của dataset (để kiểm tra)
print("Các cột trong dataset:", data.columns.tolist())

# 2. Tiền xử lý văn bản trong cột 'review'
def preprocess_text(text):
    text = str(text)            # Đảm bảo text là chuỗi
    text = text.lower()         # Chuyển về chữ thường
    # Giữ lại các ký tự chữ cái (bao gồm có dấu), số và dấu cách
    text = re.sub(r'[^a-zA-Z0-9À-ỹ\s]', '', text)
    return text

# Áp dụng tiền xử lý cho cột review
data['review'] = data['review_headline'].apply(preprocess_text)

# 3. Loại bỏ các dòng có giá trị bị thiếu
data = data.dropna(subset=['review', 'star_rating'])

# 4. Đảm bảo cột star_rating ở dạng số nguyên
data['star_rating'] = data['star_rating'].astype(int)

# Hiển thị các giá trị star_rating độc nhất
print("Các star_rating độc nhất:", data['star_rating'].unique())

# 5. Tách dữ liệu thành đặc trưng (X) và nhãn (y)
X = data['review']
y = data['star_rating']

# 6. Chia dữ liệu thành tập huấn luyện và tập kiểm tra (80% huấn luyện, 20% kiểm tra)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 7. Trích xuất đặc trưng từ văn bản bằng TF-IDF
tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# 9. Áp dụng SMOTE để cân bằng dữ liệu lớp thiểu số
X_train_dense = X_train_tfidf.toarray()  # Chuyển đổi ma trận thưa thành ma trận đặc
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_dense, y_train)

# 8. Xây dựng và huấn luyện mô hình SVM (phân loại đa lớp: 1,2,3,4,5)
svm_model = SVC(kernel='rbf', random_state=42)
svm_model.fit(X_train_tfidf, y_train)
# 10. Dự đoán nhãn cho tập kiểm tra
print("Số lượng mẫu trước SMOTE:", Counter(y_train))
print("Số lượng mẫu sau SMOTE:", Counter(y_train_resampled))

# Define the pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000)),
    ('svm', SVC(kernel='rbf', random_state=42))
])

cv_scores = cross_val_score(pipeline, X, y, cv=2, scoring='accuracy')
print("Cross-validation accuracy scores:", cv_scores)
print("Mean CV accuracy:", cv_scores.mean())
print("Mean CV accuracy:", cv_scores.mean())




# Khi chạy với côtej 'review_body' thì kết quả là:
# không có GridSearchCV + Smote + Hyperparameter, kernel: linear -> Accurancy: 0.71
# không có GridSearchCV + Smote + Hyperparameter, kernel: rbf -> Accurancy: 0.70
# Có GridSearchCV + Smote + Hyperparameter, kernel: linear class_weight='balanced' -> Accurancy: 0.6-0.64
# Có GridSearchCV + Smote + Hyperparameter, kernel: rbf class_weight='balanced' -> Accurancy: 0.64

# Khi chạy với cột 'review_headline' thì kết quả là:
# không có GridSearchCV + Smote + Hyperparameter, kernel: linear -> Accurancy: 0.77
# không có GridSearchCV + Smote + Hyperparameter, kernel: rbf -> Accurancy: 0.77
# Có GridSearchCV + Smote + Hyperparameter, kernel: rbf class_weight='balanced' -> Accurancy: 0.77




Các cột trong dataset: ['marketplace', 'customer_id', 'review_id', 'product_id', 'product_parent', 'product_title', 'product_category', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase', 'review_headline', 'review_body', 'review_date', 'sentiment']
Các star_rating độc nhất: [5 3 4 2 1]
Số lượng mẫu trước SMOTE: Counter({5: 15997, 4: 4618, 3: 1770, 1: 1365, 2: 926})
Số lượng mẫu sau SMOTE: Counter({5: 15997, 4: 15997, 3: 15997, 1: 15997, 2: 15997})
Cross-validation accuracy scores: [0.77060235 0.7818842 ]
Mean CV accuracy: 0.7762432730337807
Mean CV accuracy: 0.7762432730337807


In [3]:
# 9. Dự đoán và đánh giá mô hình
y_pred = svm_model.predict(X_test_tfidf)

print("Độ chính xác (Accuracy):", accuracy_score(y_test, y_pred), "\n")
print("Ma trận nhầm lẫn:")
print(confusion_matrix(y_test, y_pred), "\n")
print("Báo cáo phân loại:")
print(classification_report(y_test, y_pred))

Độ chính xác (Accuracy): 0.7821717990275526 

Ma trận nhầm lẫn:
[[ 180    7   35   14  107]
 [  36   51   32   22   88]
 [  34    5  181   63  163]
 [  15    5   53  502  555]
 [  16    0   12   82 3912]] 

Báo cáo phân loại:
              precision    recall  f1-score   support

           1       0.64      0.52      0.58       343
           2       0.75      0.22      0.34       229
           3       0.58      0.41      0.48       446
           4       0.73      0.44      0.55      1130
           5       0.81      0.97      0.88      4022

    accuracy                           0.78      6170
   macro avg       0.70      0.51      0.57      6170
weighted avg       0.77      0.78      0.76      6170



In [4]:

# 9. Kiểm thử với các câu thực tế
test_sentences = [
    #rating 1
    "Worst product ever! It broke within a day.",
    "Total waste of money. I regret buying this.",
    #rating 2
    "Not as described. Very disappointed with the quality." ,
    "Works okay, but expected better for the price.",
    #rating 3
    "Average product. Nothing special.",
    "It's okay, but there are better options out there.",
    #rating 4
    "Pretty good quality, just a few minor issues.",
    "Happy with my purchase! Would recommend.",
    #rating 5
    "Absolutely love it! Best purchase ever!",
    "Fantastic product, exceeded my expectations!"
]

X_test_sentences = tfidf.transform(test_sentences)
y_test_pred = svm_model.predict(X_test_sentences)

# Hiển thị kết quả
for text, pred in zip(test_sentences, y_test_pred):
    print(f"Review: {text} -> Dự đoán: {pred} sao")

Review: Worst product ever! It broke within a day. -> Dự đoán: 1 sao
Review: Total waste of money. I regret buying this. -> Dự đoán: 1 sao
Review: Not as described. Very disappointed with the quality. -> Dự đoán: 2 sao
Review: Works okay, but expected better for the price. -> Dự đoán: 3 sao
Review: Average product. Nothing special. -> Dự đoán: 5 sao
Review: It's okay, but there are better options out there. -> Dự đoán: 3 sao
Review: Pretty good quality, just a few minor issues. -> Dự đoán: 4 sao
Review: Happy with my purchase! Would recommend. -> Dự đoán: 5 sao
Review: Absolutely love it! Best purchase ever! -> Dự đoán: 5 sao
Review: Fantastic product, exceeded my expectations! -> Dự đoán: 5 sao
